In [1]:
import Augmentor as aug
import pandas as pd
import numpy as np
import os
from shutil import copyfile
from PIL import Image
import random

df_original = pd.read_csv("ODIR-5K_Training_Annotations_LR.csv")

In [2]:
df = pd.DataFrame(np.repeat(df_original.values, 2, axis=0))
df.columns = df_original.columns

In [3]:
def append_extension(fn, side):
    return f"{fn}_{side}.jpg"

df['Side'] = df.index % 2
df['ID'] = df.apply(lambda row: append_extension(row['ID'], 'left' if row['Side'] == 0 else 'right'), axis=1)

df.drop(columns=['Side'], inplace=True)

df

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O,class-left,class-right,Diseases
0,0_left.jpg,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0,['C'],['N'],['C']
1,0_right.jpg,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0,['C'],['N'],['C']
2,1_left.jpg,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0,['N'],['N'],['N']
3,1_right.jpg,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0,['N'],['N'],['N']
4,2_left.jpg,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,1,"['D', 'O']",['D'],"['D', 'O']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,4689_right.jpg,54,Male,4689_left.jpg,4689_right.jpg,mild nonproliferative retinopathy,normal fundus,0,1,0,0,0,0,0,0,['D'],['N'],['D']
6996,4690_left.jpg,57,Male,4690_left.jpg,4690_right.jpg,mild nonproliferative retinopathy,mild nonproliferative retinopathy,0,1,0,0,0,0,0,0,['D'],['D'],['D']
6997,4690_right.jpg,57,Male,4690_left.jpg,4690_right.jpg,mild nonproliferative retinopathy,mild nonproliferative retinopathy,0,1,0,0,0,0,0,0,['D'],['D'],['D']
6998,4784_left.jpg,58,Male,4784_left.jpg,4784_right.jpg,hypertensive retinopathy，age-related macular d...,hypertensive retinopathy，age-related macular d...,0,0,0,0,1,1,0,0,"['A', 'H']","['A', 'H']","['A', 'H']"


In [4]:
original_images = r"/Users/myrtekuipers/Documents/AI for Health/Year 2/AI for Medical Imaging/Challenge/ODIR_challenge/squared_and_cropped_dataset"
augmented_images = r"/Users/myrtekuipers/Documents/AI for Health/Year 2/AI for Medical Imaging/Challenge/ODIR_challenge/augmented_images"

In [5]:
#check images in original_images
original_images_list = os.listdir(augmented_images)

original_images_list

['2_left.jpg_0.jpg']

In [6]:
def augment_image(image_path):
   #p = aug.Pipeline(image_path)
    p = aug.Pipeline()
    p.add_directory(os.path.dirname(image_path))
    p.rotate(probability=0.5, max_left_rotation=15, max_right_rotation=15)
    p.flip_left_right(probability=0.5)
    p.random_contrast(probability=0.5, min_factor=0.8, max_factor=1.2)
    p.random_brightness(probability=0.5, min_factor=0.7, max_factor=1.3)
    p.random_color(probability=0.5, min_factor=0.5, max_factor=2.0)
    p.random_distortion(probability=0.5, grid_width=4, grid_height=4, magnitude=8)
    #p.random_blur(probability=0.5, min_sigma=0.1, max_sigma=2.0)
    p.process()

    # Load and return the augmented image (the first one in the pipeline)
    # augmented_image_path = p.sample_images(1)[0]

    # augmented_image = Image.open(augmented_image_path)
    augmented_image = p.augment_image(Image.open(image_path))

    return augmented_image

In [7]:
# Calculate the desired number of copies for each class
desired_class_size = 1000
class_counts = df['Diseases'].value_counts()
copy_counts = {}

# Calculate the number of copies needed for each class
for class_label, count in class_counts.items():
    if count >= desired_class_size:
        copy_counts[class_label] = 0
    else:
        copy_counts[class_label] = desired_class_size - count

# Copy and augment images to balance the classes
for class_label, copy_count in copy_counts.items():
    class_images = df[df['Diseases'] == class_label]['ID'].tolist()
    for i in range(copy_count):
        for image_name in class_images:
            source_path = os.path.join(original_images, image_name)
            destination_path = os.path.join(augmented_images, f'{image_name}_{i}.jpg')
            
            # Copy the original image to the augmented image directory
            copyfile(source_path, destination_path)
            
            # Apply augmentation to the copied image
            augmented_image = augment_image(destination_path)  
            augmented_image.save(destination_path)  # Save the augmented image

# Update the dataframe to include the augmented images
augmented_df = df.copy()
augmented_image_names = os.listdir(augmented_images)
augmented_df = augmented_df.append({'image_name': image_name, 'class': class_label}, ignore_index=True)

AttributeError: 'Pipeline' object has no attribute 'add_directory'